In [36]:
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
from tensorflow.keras import regularizers

from tensorflow.keras import layers
from tensorflow.keras import losses

from collections import Counter


import pandas as pd
import numpy as np

import sklearn


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


import seaborn as sns

import pydot



In [37]:
data_train = pd.read_csv('/content/Ben_fulltrain.csv')
data_train.head()

Unnamed: 0                                               Text  label
0           0  কবির সিংহ একেবারে জ্ঞানহীন সিনেমা .... এখানকার...      2
1           1  @ তরুন ভার্মা আপনি কোন মেয়েকে দেখেছেন যে আমি ...      2
2           2  কিছু লোক ভাল জিনিস পছন্দ করেন না তাদের সম্পর্ক...      2
3           3              ভাই আপনি এই প্রজন্মের একজন ত্রাণকর্তা      2
4           4  আমি নারীবাদী নই ... L n আমি লাস্ট স্টোরিগুলি ঘ...      2

In [38]:
data_test = pd.read_csv('/content/trac2_iben_dev.csv')

In [39]:
data_test.drop(['ID','Sub-task B'], axis=1, inplace=True)

In [40]:
NAG_test = data_test[(data_test['Sub-task A']=='NAG')]
OAG_test= data_test[(data_test['Sub-task A']=='CAG')]
CAG_test = data_test[(data_test['Sub-task A']=='OAG')]

In [41]:
import warnings as wrn
wrn.filterwarnings('ignore')

NAG_test["Sub-task A"] = 0 
#negatives_test["Sentiment"] = 0

OAG_test["Sub-task A"] = 1
#positives_test["Sentiment"] = 2

CAG_test["Sub-task A"] = 2
#neutrals_test["Sentiment"] = 1

NAG_test.head()

Text  Sub-task A
0   Ek dom sothik kotha bolecho jhekane theka uthe...           0
2                                    DARUN AKDOM THIK           0
5                                     Kadam thik dada           0
12                               Supar bolla cho dada           0
15                                              Pagle           0

In [42]:
data_test = pd.concat([NAG_test,
                  OAG_test,
                  CAG_test
                 ],axis=0)

data_test.reset_index(inplace=True)


In [43]:

data_test = data_test.rename(columns = {'Sub-task A': 'label'}, inplace = False)

In [44]:
X = data_train.Text.astype(str)
Y = data_train.label
#le = LabelEncoder()
#Y = le.fit_transform(Y)
#Y = Y.reshape(-1,1)

In [45]:
X_train,X_valid,y_train,y_valid = train_test_split(X,Y,test_size=0.15)

In [46]:
num_words = 50000

tokenizer = Tokenizer(num_words=num_words,oov_token="unk")
tokenizer.fit_on_texts(X_train)


print(str(tokenizer.texts_to_sequences(['xyz how are you'])))

[[63, 63, 216, 173]]


In [47]:

x_train = np.array( tokenizer.texts_to_sequences(X_train) )
x_valid = np.array( tokenizer.texts_to_sequences(X_valid) )
x_test  = np.array( tokenizer.texts_to_sequences(data_test['Text'].tolist()) )


In [48]:

x_train = pad_sequences(x_train, padding='post', maxlen=150)
x_valid = pad_sequences(x_valid, padding='post', maxlen=150)
x_test = pad_sequences(x_test, padding='post', maxlen=150)

In [49]:
train_labels = np.asarray(y_train )
valid_labels = np.asarray( y_valid)

test_labels = np.asarray(data_test['label'].tolist())

In [50]:

max_features =50000
embedding_dim =16
sequence_length = 150


model = tf.keras.Sequential([
    tf.keras.layers.Embedding(max_features, 64, input_length=sequence_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

num_epochs = 30
history_bi = model.fit(x_train,train_labels, epochs=num_epochs, validation_data=(x_valid, valid_labels))


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 64)           3200000   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 150, 128)          66048     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 64)                41216     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 3,311,489
Trainable params: 3,311,489
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
215/215 [==============================] - 72s 304ms/step - loss: 0.0000e+00 - accuracy: 0.5000 - v

In [51]:

x_test  = np.array( tokenizer.texts_to_sequences(data_test['Text'].tolist()) )
x_test = pad_sequences(x_test, padding='post', maxlen=150)


In [52]:
print("Generate predictions for all samples")
predictions = model.predict(x_test)

Generate predictions for all samples


In [53]:

data_test['pred']= predictions

In [54]:

data_test

index                                               Text  label  pred
0        0  Ek dom sothik kotha bolecho jhekane theka uthe...      0   0.0
1        2                                   DARUN AKDOM THIK      0   0.0
2        5                                    Kadam thik dada      0   0.0
3       12                               Supar bolla cho dada      0   1.0
4       15                                              Pagle      0   1.0
..     ...                                                ...    ...   ...
952    935                         Vikari ranu tor gaan faltu      2   1.0
953    938                             Khanki magi ranu pagli      2   1.0
954    941                              কুকুরের বাচ্চা শালারা      2   1.0
955    948    জংলি হাতি,সব।বুর মাগির গুর দুধ জত পারিস তত চোদ।      2   1.0
956    950  @Habibul Sekh মোল্লা খানকির বাচ্চা মুখ সামলে ক...      2   1.0

[957 rows x 4 columns]

In [55]:

labels = [0,1,2]
    
print(classification_report(data_test['label'].tolist(),data_test['pred'].tolist(),labels=labels))

              precision    recall  f1-score   support

           0       0.77      0.49      0.60       522
           1       0.27      0.77      0.40       218
           2       0.00      0.00      0.00       217

    accuracy                           0.44       957
   macro avg       0.35      0.42      0.33       957
weighted avg       0.48      0.44      0.42       957

